In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# choose default setting or AMD setting

In [3]:
# default setting
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16

In [2]:
# AMD setting
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.environ["PLAIDML_EXPERIMENTAL"] = "1"
os.environ["PLAIDML_DEVICE_IDS"] = "opencl_amd_ellesmere.0"

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

Using plaidml.keras.backend backend.


In [3]:
# start after setting

In [6]:
# you can change this values
class_names = ['Non', 'Cut', 'Dissolve', 'Fade']
batch_size = 10
train_steps = 120 # = total_train_image / batch_size
valid_steps = 40 # = total_valid_image / batch_size
epochs = 10

In [7]:
# 
train_image_generator = ImageDataGenerator(rescale=1./255)
valid_image_generator = ImageDataGenerator(rescale=1./255)
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory='train',
                                                           shuffle=True,
                                                           target_size=(224, 224),
                                                           class_mode='binary')
valid_data_gen = valid_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory='valid',
                                                           target_size=(224, 224),
                                                           class_mode='binary')

Found 1200 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [8]:
model = VGG16(weights=None, classes=len(class_names))
#model.summary()

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=valid_data_gen,
    validation_steps=valid_steps
)

Epoch 1/10


INFO:plaidml:Analyzing Ops: 126 of 650 operations complete


120/120 [==============================] - 309s 3s/step - loss: 1.8260 - acc: 0.2458 - val_loss: 1.3864 - val_acc: 0.2500
Epoch 2/10
120/120 [==============================] - 280s 2s/step - loss: 1.3870 - acc: 0.2275 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 3/10
120/120 [==============================] - 280s 2s/step - loss: 1.3866 - acc: 0.2275 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 4/10
120/120 [==============================] - 280s 2s/step - loss: 1.3866 - acc: 0.2350 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 5/10
120/120 [==============================] - 280s 2s/step - loss: 1.3865 - acc: 0.2250 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 6/10
120/120 [==============================] - 281s 2s/step - loss: 1.3865 - acc: 0.2250 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 7/10
120/120 [==============================] - 281s 2s/step - loss: 1.3866 - acc: 0.2167 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 8/10
120/120 [==============================] - 281s 2s/step - loss: 1.386